In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.pipeline import make_pipeline

In [57]:
import sklearn
import sklearn.datasets as datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder



In [49]:
data_home = datasets.fetch_openml(name='credit-g')      # Загрузка датасета с OpenML
Predictors_home = pd.DataFrame(data_home.data, columns=data_home.feature_names)
Target_home = data_home.get('target')

c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(


In [59]:
categorical_features = ['checking_status', 'credit_history', 'employment', 'foreign_worker',
'housing',
'job',
'other_parties',
'other_payment_plans',
'own_telephone',
'personal_status',
'property_magnitude',
'purpose',
'savings_status']

numeric_features = [feature for feature in data_home.feature_names if feature not in categorical_features]

In [56]:
predictors_home = pd.get_dummies(Predictors_home, columns = categorical_features)

le = LabelEncoder()
y = le.fit_transform(Target_home)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(predictors_home, y)

In [35]:
logistic_regression = LogisticRegression().fit(X_train, y_train)

y_train_approx = logistic_regression.predict(X_train)
f1_score(y_train, y_train_approx, average = "macro")

c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7064484638620792

KNN

In [45]:
from sklearn.neighbors import KNeighborsClassifier

model_KNN = KNeighborsClassifier()
params = {"n_neighbors": range(10, 20)}

grid_search = GridSearchCV(estimator = model_KNN, 
                           param_grid = params, 
                           cv = 5, 
                           scoring = "f1_macro")

grid_search.fit(X_train, y_train)
grid_search.best_score_

0.5290278498054857

In [80]:
preds_knn = grid_search.predict(X_test)
print(classification_report(y_test, preds_knn))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        76
           1       0.70      1.00      0.82       174

    accuracy                           0.70       250
   macro avg       0.35      0.50      0.41       250
weighted avg       0.48      0.70      0.57       250



c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

SVM

In [67]:
Scaler = StandardScaler()

X_val_scaled = X_train
X_val_scaled[numeric_features] = Scaler.fit_transform(X_val_scaled[numeric_features])

X_val_scaled1 = X_test
X_val_scaled1[numeric_features] = Scaler.fit_transform(X_val_scaled1[numeric_features])


In [78]:
line_param = np.linspace(-10, 5, 20)
parameters = {'kernel':('linear', 'rbf', "poly", "sigmoid"), 'C':line_param}
model = SVC()


grid_svc = GridSearchCV(estimator = model, param_grid = parameters, cv=5)
grid_svc.fit(X_val_scaled, y_train)
grid_svc.best_score_

c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
260 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
260 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\vasil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py", line 251, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "c:\Users\vasil\AppData\Local\Programs\Python\Python39

0.7826666666666667

In [81]:
preds_svm = grid_svc.predict(X_val_scaled1)
print(classification_report(y_test, preds_svm))

              precision    recall  f1-score   support

           0       0.70      0.43      0.54        76
           1       0.79      0.92      0.85       174

    accuracy                           0.77       250
   macro avg       0.75      0.68      0.69       250
weighted avg       0.76      0.77      0.75       250

